In [1]:
import cv2
import numpy as np
import mediapipe as mp
import os
import time

# Initialize MediaPipe Face Mesh solution
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Function to calculate the Eye Aspect Ratio (EAR)
def calculate_ear(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Threshold for detecting closed eyes
EAR_THRESHOLD = 0.24
CONSECUTIVE_FRAMES = 10

# Create directories for saving images
base_dir = 'dataset'
drowsy_dir = os.path.join(base_dir, 'drowsy')
alert_dir = os.path.join(base_dir, 'alert')
os.makedirs(drowsy_dir, exist_ok=True)
os.makedirs(alert_dir, exist_ok=True)

# Start video capture
cap = cv2.VideoCapture(0)

# Initialize variables for tracking eye closure
consecutive_frames_blinking = 0
eye_closed = False
image_count_drowsy = 0
image_count_alert = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to get facial landmarks
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Extract landmarks for the left and right eye
            left_eye = np.array([[face_landmarks.landmark[i].x, face_landmarks.landmark[i].y] 
                                for i in [33, 160, 158, 133, 153, 144]])
            right_eye = np.array([[face_landmarks.landmark[i].x, face_landmarks.landmark[i].y] 
                                 for i in [362, 385, 387, 263, 373, 380]])

            # Convert normalized coordinates to pixel coordinates
            h, w, _ = frame.shape
            left_eye = (left_eye * [w, h]).astype(int)
            right_eye = (right_eye * [w, h]).astype(int)

            # Calculate EAR for both eyes
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)

            # Average EAR
            ear = (left_ear + right_ear) / 2.0

            # Check if EAR is below the threshold
            if ear < EAR_THRESHOLD:
                consecutive_frames_blinking += 1
                if consecutive_frames_blinking >= CONSECUTIVE_FRAMES:
                    eye_closed = True
            else:
                consecutive_frames_blinking = 0
                eye_closed = False

            # Display EAR and eye status
            cv2.putText(frame, f'EAR: {ear:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            if eye_closed:
                cv2.putText(frame, 'Eyes Closed', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                image_path = os.path.join(drowsy_dir, f'drowsy_{image_count_drowsy}.jpg')
                image_count_drowsy += 1
            else:
                image_path = os.path.join(alert_dir, f'alert_{image_count_alert}.jpg')
                image_count_alert += 1

            # Save image based on eye status
            if eye_closed or ear > EAR_THRESHOLD:
                cv2.imwrite(image_path, frame)

            # Draw the eyes
            for (x, y) in left_eye:
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)
            for (x, y) in right_eye:
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

    # Display the frame
    cv2.imshow('Frame', frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


C:\Users\fox-trot\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [1]:
import cv2
import numpy as np
import mediapipe as mp
import os
import time
import uuid

# Initialize MediaPipe Face Mesh solution
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Function to calculate the Eye Aspect Ratio (EAR)
def calculate_ear(eye):
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Threshold for detecting closed eyes
EAR_THRESHOLD = 0.24
CONSECUTIVE_FRAMES = 10
IMAGE_CAPTURE_RATE = 4 

x1, y1 = 150, 150
x2, y2 = 550, 550

base_dir = 'dataset'
drowsy_dir = os.path.join(base_dir, 'drowsy')
alert_dir = os.path.join(base_dir, 'alert')
os.makedirs(drowsy_dir, exist_ok=True)
os.makedirs(alert_dir, exist_ok=True)

# Start video capture
cap = cv2.VideoCapture(0)

# Initialize variables for tracking eye closure
consecutive_frames_blinking = 0
eye_closed = False

# Time interval between captures
time_interval = 1.0 / IMAGE_CAPTURE_RATE
last_capture_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to get facial landmarks
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Extract landmarks for the left and right eye
            left_eye = np.array([[face_landmarks.landmark[i].x, face_landmarks.landmark[i].y] 
                                for i in [33, 160, 158, 133, 153, 144]])
            right_eye = np.array([[face_landmarks.landmark[i].x, face_landmarks.landmark[i].y] 
                                 for i in [362, 385, 387, 263, 373, 380]])

            # Convert normalized coordinates to pixel coordinates
            h, w, _ = frame.shape
            left_eye = (left_eye * [w, h]).astype(int)
            right_eye = (right_eye * [w, h]).astype(int)

            # Calculate EAR for both eyes
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)

            # Average EAR
            ear = (left_ear + right_ear) / 2.0

            # Check if EAR is below the threshold
            if ear < EAR_THRESHOLD:
                consecutive_frames_blinking += 1
                if consecutive_frames_blinking >= CONSECUTIVE_FRAMES:
                    eye_closed = True
            else:
                consecutive_frames_blinking = 0
                eye_closed = False

            # Crop the frame using specified coordinates
            cropped_frame = frame[y1:y2, x1:x2]

            # Save image and EAR value based on eye status if it's time to capture
            current_time = time.time()
            if current_time - last_capture_time >= time_interval:
                last_capture_time = current_time

                # Generate a unique directory for each capture
                uuid_str = str(uuid.uuid4())
                if eye_closed:
                    directory = os.path.join(drowsy_dir, uuid_str)
                else:
                    directory = os.path.join(alert_dir, uuid_str)
                
                os.makedirs(directory, exist_ok=True)

                # Save the image
                image_path = os.path.join(directory, 'image.jpg')
                cv2.imwrite(image_path, cropped_frame)

                # Save the EAR value to a text file
                ear_path = os.path.join(directory, 'EAR.txt')
                with open(ear_path, 'w') as f:
                    f.write(f'{ear:.2f}')

    # Display the cropped frame (optional)
    cv2.imshow('Frame', cropped_frame)

    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


C:\Users\fox-trot\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
